### Talk Time Data Prepare

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

# word usage
min_word_use_num = 1
max_word_use_num = 6

# youtube
sample_num = 1  # 6
time_shift = 0.6

In [4]:
def word_in_wordgroup_simple(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column row result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"search_string",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [5]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [6]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")\n
    df_search_result is dataframe and "search_string", "start_time", "end_time", "sentence", "video_id" are its columns
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [7]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [8]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_threegram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [9]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Data/3-Talk Time Data Prepare"

Path(path).mkdir(parents=True, exist_ok=True)

In [10]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
331769,oorlogsgezichten,5
331770,opdrachtenlijst,5
331771,verlsaafde,5
331772,oxidatieproces,5


In [11]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [12]:
df_word_select

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
195,genoeg,353128
196,wist,352948
197,oh,351988
198,klaar,350506


In [13]:
df_word_select.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word.xlsx", index=False)

In [14]:
word_list = df_word_select["word"].to_list()
len(word_list)

200

In [31]:
df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
df_twogram_sent

,twogram,frequency
0,kom op,184578
1,dank je,127178
2,dank u,64014
3,wacht even,63826
4,echt waar,59238
...,...,...
328541,zesentwintig horizontaal,3
328542,volledig gedisciplineerd,3
328543,volledig geconcentreerd,3
328544,hero netegengif,3


In [32]:
# Twogram Func
d_list2  = df_twogram_sent.iloc[:,0].values.tolist()

resultlist2 = []
manager = multiprocessing.Manager()
resultlist2 = manager.list()

def word_in_wordgroup(d_list2):
    mergelist2 = []
    try:
        word = d_list2.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist2.append(word[j])
            if len(mergelist2) == len(word):
                    resultlist2.append(d_list2)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list2) # string_word liste

result_list2 = list(resultlist2)
df_result2 = pd.DataFrame(result_list2, columns=[0])  # add columns parameter for empty result
df_result2 = df_result2.rename(columns = {0: "twogram"})
df_merge2 = pd.merge(df_result2, df_twogram_sent, how="left", on="twogram")
df_merge_result2 = df_merge2.sort_values(by="frequency", ascending=False)
df_merge_result2.drop_duplicates(inplace=True)
df_merge_result2.reset_index(drop=True, inplace=True)
df_twogram_select = df_merge_result2
df_twogram_select

,twogram,frequency
0,kom op,184578
1,dank je,127178
2,dank u,64014
3,wacht even,63826
4,echt waar,59238
...,...,...
6564,als of,3
6565,praten wel,3
6566,jullie kijk,3
6567,van het,3


In [33]:
df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
df_threegram_sent

,threegram,frequency
0,het spijt me,132758
1,wat is er,106497
2,ik weet het,67518
3,wat is dat,62768
4,wat doe je,60323
...,...,...
550436,vliegen onbekend aantal,3
550437,vliegen obstakel afnemen,3
550438,voor achtduizend euro,3
550439,voor de kijkcijfers,3


In [34]:
# Threegram Func
d_list3  = df_threegram_sent.iloc[:,0].values.tolist()

resultlist3 = []
manager = multiprocessing.Manager()
resultlist3 = manager.list()

def word_in_wordgroup(d_list3):
    mergelist3 = []
    try:
        word = d_list3.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist3.append(word[j])
            if len(mergelist3) == len(word):
                    resultlist3.append(d_list3)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list3) # string_word liste

result_list3 = list(resultlist3)
df_result3 = pd.DataFrame(result_list3, columns=[0])  # add columns parameter for empty result
df_result3 = df_result3.rename(columns = {0: "threegram"})
df_merge3 = pd.merge(df_result3, df_threegram_sent, how="left", on="threegram")
df_merge_result3 = df_merge3.sort_values(by="frequency", ascending=False)
df_merge_result3.drop_duplicates(inplace=True)
df_merge_result3.reset_index(drop=True, inplace=True)
df_threegram_select = df_merge_result3
df_threegram_select

,threegram,frequency
0,het spijt me,132758
1,wat is er,106497
2,ik weet het,67518
3,wat is dat,62768
4,wat doe je,60323
...,...,...
29304,maar wij allemaal,3
29305,nee dit keer,3
29306,h h nu,3
29307,staat er waarom,3


In [16]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.000,00:00:02.501,dit is de pangale v,BoaYsdPtJYA
1,00:00:04.501,00:00:05.434,unreal,BoaYsdPtJYA
2,00:00:08.518,00:00:13.066,dit is de eerste keer ik rijd in een supersport,BoaYsdPtJYA
3,00:00:13.066,00:00:14.316,ik heb dit in mijn hele leven,BoaYsdPtJYA
4,00:00:14.316,00:00:15.889,nog nooit eerder gedaan,BoaYsdPtJYA
...,...,...,...,...
252485,00:01:37.135,00:01:47.040,soms zitten de volwassenen met natte ogen te k...,MZUDbceIw4E
252486,00:01:47.040,00:01:49.340,dit ziet er al was super indrukwekkend uit en,MZUDbceIw4E
252487,00:01:49.340,00:01:53.600,het is nog maar een voorproefje dus kom zeker ...,MZUDbceIw4E
252488,00:01:53.600,00:01:57.914,tijdsblok te kiezen en zeker ook op tijd te ko...,MZUDbceIw4E


In [17]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [18]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,0.000,2.501,dit is de pangale v,BoaYsdPtJYA
1,4.501,5.434,unreal,BoaYsdPtJYA
2,8.518,13.066,dit is de eerste keer ik rijd in een supersport,BoaYsdPtJYA
3,13.066,14.316,ik heb dit in mijn hele leven,BoaYsdPtJYA
4,14.316,15.889,nog nooit eerder gedaan,BoaYsdPtJYA
...,...,...,...,...
252485,97.135,107.040,soms zitten de volwassenen met natte ogen te k...,MZUDbceIw4E
252486,107.040,109.340,dit ziet er al was super indrukwekkend uit en,MZUDbceIw4E
252487,109.340,113.600,het is nog maar een voorproefje dus kom zeker ...,MZUDbceIw4E
252488,113.600,117.914,tijdsblok te kiezen en zeker ook op tijd te ko...,MZUDbceIw4E


In [19]:
df_sentence_link = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Result/2-Adjust Word Group In Youtube Sentence Word Usage Analysis/{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Sample_Manuel.xlsx")
df_sentence_link = df_sentence_link.iloc[:,[0,1,2,3,4,5]]
df_sentence_link

,search_string,start_time,end_time,sentence,video_id,video_url
0,van wat zou dat nou voor de,141,143,maar ik heb wel beter nagedacht van wat zou da...,JUQTVKIBFns,https://www.youtube.com/watch?v=JUQTVKIBFns&t=...
1,en we daar hebben wij een,167,168,en we daar hebben wij een efteling variant van...,JUQTVKIBFns,https://www.youtube.com/watch?v=JUQTVKIBFns&t=...
2,en we hebben weer een heel,332,334,het staat natuurlijk gewoon in de efteling en ...,JUQTVKIBFns,https://www.youtube.com/watch?v=JUQTVKIBFns&t=...
3,me af of dat wel goed zou komen,230,232,dus ik vroeg me af of dat wel goed zou komen,UltPsmBVTV0,https://www.youtube.com/watch?v=UltPsmBVTV0&t=...
4,dus ja er zijn veel mensen daar,480,483,ah het is dit hele complex dus ja er zijn veel...,UltPsmBVTV0,https://www.youtube.com/watch?v=UltPsmBVTV0&t=...
5,kunnen doen ja dan moet de,194,195,maar je moet het snel kunnen doen ja dan moet ...,4YSldeXmuXw,https://www.youtube.com/watch?v=4YSldeXmuXw&t=...
6,we hebben hem toch in een,274,275,we hebben hem toch in een korte deadline neerg...,KmcDJ7GyLNk,https://www.youtube.com/watch?v=KmcDJ7GyLNk&t=...
7,maar door me in hem te,102,103,ik kende alleen maar het verhaal van max amp m...,IJ2LOTPGSvA,https://www.youtube.com/watch?v=IJ2LOTPGSvA&t=...
8,en moeten wij hier weg zijn,65,67,en moeten wij hier weg zijn,b_HmOS8lNl4,https://www.youtube.com/watch?v=b_HmOS8lNl4&t=65s
9,dan moeten wij hier weg zijn anders,66,68,dan moeten wij hier weg zijn anders krijgen we...,b_HmOS8lNl4,https://www.youtube.com/watch?v=b_HmOS8lNl4&t=66s


In [23]:
twogram_set = set()
threegram_set = set()
for sent in df_sentence_link["search_string"]:
    twogram_zip = ngrams(sent.split(), 2)
    for two in twogram_zip:
        twogram = " ".join(two)
        twogram_set.add(f"{twogram}") 
    threegram_zip = ngrams(sent.split(), 3)
    for three in threegram_zip:
        threegram = " ".join(three)
        threegram_set.add(f"{threegram}")

In [24]:
len(twogram_set)

103

In [25]:
len(threegram_set)

94

In [26]:
twogram_list = list(twogram_set)
#twogram_list

In [27]:
threegram_list = list(threegram_set)
#threegram_list

In [28]:
df_twogram_for_link = df_twogram_select[df_twogram_select["twogram"].isin(twogram_list)]
df_twogram_for_link.reset_index(drop=True, inplace=True)
df_twogram_for_link

,twogram,frequency
0,weet je,12153
1,hoe dan,9794
2,ik ga,8296
3,het is,7417
4,dat is,5448
...,...,...
60,wel even,5
61,er aan,5
62,waar ik,4
63,denk dat,4


In [29]:
df_twogram_selected_for_link = word_usage_result(word_list, df_twogram_for_link, "twogram", "frequency", min_word_use_num, max_word_use_num)
df_twogram_selected_for_link 

,twogram,frequency
0,weet je,12153
1,hoe dan,9794
2,ik ga,8296
3,het is,7417
4,dat is,5448
5,ik ben,5395
6,dit is,3284
7,dat kan,2892
8,ik denk,2821
9,ik heb,2400


In [30]:
df_threegram_for_link = df_threegram_select[df_threegram_select["threegram"].isin(threegram_list)]
df_threegram_for_link.reset_index(drop=True, inplace=True)
df_threegram_for_link

,threegram,frequency
0,weet je wat,31426
1,en dat is,4494
2,hoe dan ook,3855
3,ik denk dat,666
4,dat is wat,161
5,ik ben ik,142
6,je weet wat,93
7,nog veel meer,90
8,wat het is,78
9,waar ik ben,75


In [31]:
df_threegram_selected_for_link = word_usage_result(word_list, df_threegram_for_link, "threegram", "frequency", min_word_use_num, max_word_use_num)
df_threegram_selected_for_link 

,threegram,frequency
0,weet je wat,31426
1,en dat is,4494
2,hoe dan ook,3855
3,ik denk dat,666
4,dat is wat,161
5,ik ben ik,142
6,je weet wat,93
7,nog veel meer,90
8,wat het is,78
9,waar ik ben,75


In [32]:
df_twogram_search_result = word_group_youtube(df_youtube_sentence, df_twogram_selected_for_link["twogram"], "sentence", sample_num)
df_twogram_search_result

,search_string,start_time,end_time,sentence,video_id
0,weet je,1434.693,1436.174,weet je wat lopen we daar binnen,fPzfhneNmcc
1,hoe dan,135.380,140.690,stukje belgische geschiedenis en hoe dan de ch...,Dd6N0ZdQTvc
2,ik ga,1137.600,1147.320,ik ga daarop klikken en dan krijg ik de mogeli...,p4Xe249nsTU
3,het is,68.443,73.575,want het is groen dat deze wijken met elkaar k...,Kh9znOzlKPw
4,dat is,215.260,216.900,dat is eigenlijk een levenswerk voor u,kRLm5pQi_xE
5,ik ben,59.850,68.790,nooit in positieve zin en daarom ben ik op een...,8oxf7nVWqIE
6,dit is,1376.300,1384.300,moet ik zeggen dat als ik even voor ons beide ...,XhcjYIknnfA
7,dat kan,131.870,137.030,als je vragen hebt of je bent een beetje bang ...,rWNpd_54cqM
8,ik denk,563.659,565.659,ik denk daar is men dan ook wel weer mee bezig,qmqWWRrEvZk
9,ik heb,125.680,136.110,hoe kijk jij daarnaar amber ik heb die spotjes...,Ud7n8yMn_h8


In [33]:
df_word_group_time_loc_twogram_result = word_group_time_loc(df_twogram_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_twogram_result

,search_string,start_time,end_time,sentence,video_id
0,weet je,1434.693000,1435.063250,weet je wat lopen we daar binnen,fPzfhneNmcc
1,hoe dan,137.586753,138.207403,stukje belgische geschiedenis en hoe dan de ch...,Dd6N0ZdQTvc
2,ik ga,1137.600000,1138.005000,ik ga daarop klikken en dan krijg ik de mogeli...,p4Xe249nsTU
3,het is,68.823148,69.583444,want het is groen dat deze wijken met elkaar k...,Kh9znOzlKPw
4,dat is,215.260000,215.562105,dat is eigenlijk een levenswerk voor u,kRLm5pQi_xE
5,ik ben,68.109783,68.790000,nooit in positieve zin en daarom ben ik op een...,8oxf7nVWqIE
6,dit is,1381.564516,1381.977419,moet ik zeggen dat als ik even voor ons beide ...,XhcjYIknnfA
7,dat kan,134.736667,135.310000,als je vragen hebt of je bent een beetje bang ...,rWNpd_54cqM
8,ik denk,563.659000,564.006826,ik denk daar is men dan ook wel weer mee bezig,qmqWWRrEvZk
9,ik heb,129.761304,130.970580,hoe kijk jij daarnaar amber ik heb die spotjes...,Ud7n8yMn_h8


In [34]:
df_word_group_time_loc_twogram_result.start_time = df_word_group_time_loc_twogram_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_twogram_result.end_time = df_word_group_time_loc_twogram_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_twogram_result

,search_string,start_time,end_time,sentence,video_id
0,weet je,1434.093000,1435.663250,weet je wat lopen we daar binnen,fPzfhneNmcc
1,hoe dan,136.986753,138.807403,stukje belgische geschiedenis en hoe dan de ch...,Dd6N0ZdQTvc
2,ik ga,1137.000000,1138.605000,ik ga daarop klikken en dan krijg ik de mogeli...,p4Xe249nsTU
3,het is,68.223148,70.183444,want het is groen dat deze wijken met elkaar k...,Kh9znOzlKPw
4,dat is,214.660000,216.162105,dat is eigenlijk een levenswerk voor u,kRLm5pQi_xE
5,ik ben,67.509783,69.390000,nooit in positieve zin en daarom ben ik op een...,8oxf7nVWqIE
6,dit is,1380.964516,1382.577419,moet ik zeggen dat als ik even voor ons beide ...,XhcjYIknnfA
7,dat kan,134.136667,135.910000,als je vragen hebt of je bent een beetje bang ...,rWNpd_54cqM
8,ik denk,563.059000,564.606826,ik denk daar is men dan ook wel weer mee bezig,qmqWWRrEvZk
9,ik heb,129.161304,131.570580,hoe kijk jij daarnaar amber ik heb die spotjes...,Ud7n8yMn_h8


In [35]:
df_word_group_time_loc_twogram_result.start_time = df_word_group_time_loc_twogram_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_twogram_result.end_time = df_word_group_time_loc_twogram_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_twogram_result 

,search_string,start_time,end_time,sentence,video_id
0,weet je,1434,1436,weet je wat lopen we daar binnen,fPzfhneNmcc
1,hoe dan,137,139,stukje belgische geschiedenis en hoe dan de ch...,Dd6N0ZdQTvc
2,ik ga,1137,1139,ik ga daarop klikken en dan krijg ik de mogeli...,p4Xe249nsTU
3,het is,68,70,want het is groen dat deze wijken met elkaar k...,Kh9znOzlKPw
4,dat is,215,216,dat is eigenlijk een levenswerk voor u,kRLm5pQi_xE
5,ik ben,68,69,nooit in positieve zin en daarom ben ik op een...,8oxf7nVWqIE
6,dit is,1381,1383,moet ik zeggen dat als ik even voor ons beide ...,XhcjYIknnfA
7,dat kan,134,136,als je vragen hebt of je bent een beetje bang ...,rWNpd_54cqM
8,ik denk,563,565,ik denk daar is men dan ook wel weer mee bezig,qmqWWRrEvZk
9,ik heb,129,132,hoe kijk jij daarnaar amber ik heb die spotjes...,Ud7n8yMn_h8


In [36]:
df_word_group_time_loc_twogram_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_twogram_result['video_id'].map(str)+"&t="+df_word_group_time_loc_twogram_result['start_time'].map(str)+"s"
df_word_group_time_loc_twogram_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,weet je,1434,1436,weet je wat lopen we daar binnen,fPzfhneNmcc,https://www.youtube.com/watch?v=fPzfhneNmcc&t=...
1,hoe dan,137,139,stukje belgische geschiedenis en hoe dan de ch...,Dd6N0ZdQTvc,https://www.youtube.com/watch?v=Dd6N0ZdQTvc&t=...
2,ik ga,1137,1139,ik ga daarop klikken en dan krijg ik de mogeli...,p4Xe249nsTU,https://www.youtube.com/watch?v=p4Xe249nsTU&t=...
3,het is,68,70,want het is groen dat deze wijken met elkaar k...,Kh9znOzlKPw,https://www.youtube.com/watch?v=Kh9znOzlKPw&t=68s
4,dat is,215,216,dat is eigenlijk een levenswerk voor u,kRLm5pQi_xE,https://www.youtube.com/watch?v=kRLm5pQi_xE&t=...
5,ik ben,68,69,nooit in positieve zin en daarom ben ik op een...,8oxf7nVWqIE,https://www.youtube.com/watch?v=8oxf7nVWqIE&t=68s
6,dit is,1381,1383,moet ik zeggen dat als ik even voor ons beide ...,XhcjYIknnfA,https://www.youtube.com/watch?v=XhcjYIknnfA&t=...
7,dat kan,134,136,als je vragen hebt of je bent een beetje bang ...,rWNpd_54cqM,https://www.youtube.com/watch?v=rWNpd_54cqM&t=...
8,ik denk,563,565,ik denk daar is men dan ook wel weer mee bezig,qmqWWRrEvZk,https://www.youtube.com/watch?v=qmqWWRrEvZk&t=...
9,ik heb,129,132,hoe kijk jij daarnaar amber ik heb die spotjes...,Ud7n8yMn_h8,https://www.youtube.com/watch?v=Ud7n8yMn_h8&t=...


In [37]:
df_word_group_time_loc_twogram_result.to_excel(f"{lang_folder.capitalize()}_Twogram_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx", index=False) 

In [38]:
df_threegram_search_result = word_group_youtube(df_youtube_sentence, df_threegram_selected_for_link["threegram"], "sentence", sample_num)
df_threegram_search_result

,search_string,start_time,end_time,sentence,video_id
0,weet je wat,4403.560,4409.230,meneer weet u dat u meneer weet u zo maar weet...,lApJy0bJNk0
1,en dat is,2592.610,2599.600,van opstellen en dat is ook de reden dat wij d...,idTEcq_P1WY
2,hoe dan ook,5212.123,5214.102,en des tilleuls hoe dan ook opgenomen te,R4Q1cNbNKU0
3,ik denk dat,226.139,231.000,in het onderwijs bijvoorbeeld en ik denk dat d...,S0MhQFLZGYg
4,dat is wat,58.369,64.559,intense weekenden dat is wat ik zeggen,rdLMBm1Rtcs
5,ik ben ik,428.780,432.920,alleen ben ik nog wel heel lang bezig en ik be...,NxX9X3tkSXw
6,je weet wat,264.169,268.610,je weet wat te doen dankjewel,tHFynkakpVk
7,nog veel meer,13.670,16.370,wie er belt waarom diegene belt en nog veel meer,Hvc226NRmSQ
8,wat het is,3926.060,3933.800,mensen willen aansluiten nou weet je wat het i...,lApJy0bJNk0
9,waar ik ben,116.302,119.175,waar ik ben ik zou mijn fiets moeten en ik ga ...,BoaYsdPtJYA


In [39]:
df_word_group_time_loc_threegram_result = word_group_time_loc(df_threegram_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_threegram_result

,search_string,start_time,end_time,sentence,video_id
0,weet je wat,4406.327500,4407.205000,meneer weet u dat u meneer weet u zo maar weet...,lApJy0bJNk0
1,en dat is,2593.587097,2594.413871,van opstellen en dat is ook de reden dat wij d...,idTEcq_P1WY
2,hoe dan ook,5212.865125,5213.508300,en des tilleuls hoe dan ook opgenomen te,R4Q1cNbNKU0
3,ik denk dat,228.495848,229.453318,in het onderwijs bijvoorbeeld en ik denk dat d...,S0MhQFLZGYg
4,dat is wat,61.138211,63.092947,intense weekenden dat is wat ik zeggen,rdLMBm1Rtcs
5,ik ben ik,430.661818,431.179318,alleen ben ik nog wel heel lang bezig en ik be...,NxX9X3tkSXw
6,je weet wat,264.169000,266.006655,je weet wat te doen dankjewel,tHFynkakpVk
7,nog veel meer,15.582500,16.370000,wie er belt waarom diegene belt en nog veel meer,Hvc226NRmSQ
8,wat het is,3928.903265,3929.851020,mensen willen aansluiten nou weet je wat het i...,lApJy0bJNk0
9,waar ik ben,116.302000,116.928836,waar ik ben ik zou mijn fiets moeten en ik ga ...,BoaYsdPtJYA


In [40]:
df_word_group_time_loc_threegram_result.start_time = df_word_group_time_loc_threegram_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_threegram_result.end_time = df_word_group_time_loc_threegram_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_threegram_result

,search_string,start_time,end_time,sentence,video_id
0,weet je wat,4405.727500,4407.805000,meneer weet u dat u meneer weet u zo maar weet...,lApJy0bJNk0
1,en dat is,2592.987097,2595.013871,van opstellen en dat is ook de reden dat wij d...,idTEcq_P1WY
2,hoe dan ook,5212.265125,5214.108300,en des tilleuls hoe dan ook opgenomen te,R4Q1cNbNKU0
3,ik denk dat,227.895848,230.053318,in het onderwijs bijvoorbeeld en ik denk dat d...,S0MhQFLZGYg
4,dat is wat,60.538211,63.692947,intense weekenden dat is wat ik zeggen,rdLMBm1Rtcs
5,ik ben ik,430.061818,431.779318,alleen ben ik nog wel heel lang bezig en ik be...,NxX9X3tkSXw
6,je weet wat,263.569000,266.606655,je weet wat te doen dankjewel,tHFynkakpVk
7,nog veel meer,14.982500,16.970000,wie er belt waarom diegene belt en nog veel meer,Hvc226NRmSQ
8,wat het is,3928.303265,3930.451020,mensen willen aansluiten nou weet je wat het i...,lApJy0bJNk0
9,waar ik ben,115.702000,117.528836,waar ik ben ik zou mijn fiets moeten en ik ga ...,BoaYsdPtJYA


In [41]:
df_word_group_time_loc_threegram_result.start_time = df_word_group_time_loc_threegram_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_threegram_result.end_time = df_word_group_time_loc_threegram_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_threegram_result 

,search_string,start_time,end_time,sentence,video_id
0,weet je wat,4406,4408,meneer weet u dat u meneer weet u zo maar weet...,lApJy0bJNk0
1,en dat is,2593,2595,van opstellen en dat is ook de reden dat wij d...,idTEcq_P1WY
2,hoe dan ook,5212,5214,en des tilleuls hoe dan ook opgenomen te,R4Q1cNbNKU0
3,ik denk dat,228,230,in het onderwijs bijvoorbeeld en ik denk dat d...,S0MhQFLZGYg
4,dat is wat,61,64,intense weekenden dat is wat ik zeggen,rdLMBm1Rtcs
5,ik ben ik,430,432,alleen ben ik nog wel heel lang bezig en ik be...,NxX9X3tkSXw
6,je weet wat,264,267,je weet wat te doen dankjewel,tHFynkakpVk
7,nog veel meer,15,17,wie er belt waarom diegene belt en nog veel meer,Hvc226NRmSQ
8,wat het is,3928,3930,mensen willen aansluiten nou weet je wat het i...,lApJy0bJNk0
9,waar ik ben,116,118,waar ik ben ik zou mijn fiets moeten en ik ga ...,BoaYsdPtJYA


In [42]:
df_word_group_time_loc_threegram_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_threegram_result['video_id'].map(str)+"&t="+df_word_group_time_loc_threegram_result['start_time'].map(str)+"s"
df_word_group_time_loc_threegram_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,weet je wat,4406,4408,meneer weet u dat u meneer weet u zo maar weet...,lApJy0bJNk0,https://www.youtube.com/watch?v=lApJy0bJNk0&t=...
1,en dat is,2593,2595,van opstellen en dat is ook de reden dat wij d...,idTEcq_P1WY,https://www.youtube.com/watch?v=idTEcq_P1WY&t=...
2,hoe dan ook,5212,5214,en des tilleuls hoe dan ook opgenomen te,R4Q1cNbNKU0,https://www.youtube.com/watch?v=R4Q1cNbNKU0&t=...
3,ik denk dat,228,230,in het onderwijs bijvoorbeeld en ik denk dat d...,S0MhQFLZGYg,https://www.youtube.com/watch?v=S0MhQFLZGYg&t=...
4,dat is wat,61,64,intense weekenden dat is wat ik zeggen,rdLMBm1Rtcs,https://www.youtube.com/watch?v=rdLMBm1Rtcs&t=61s
5,ik ben ik,430,432,alleen ben ik nog wel heel lang bezig en ik be...,NxX9X3tkSXw,https://www.youtube.com/watch?v=NxX9X3tkSXw&t=...
6,je weet wat,264,267,je weet wat te doen dankjewel,tHFynkakpVk,https://www.youtube.com/watch?v=tHFynkakpVk&t=...
7,nog veel meer,15,17,wie er belt waarom diegene belt en nog veel meer,Hvc226NRmSQ,https://www.youtube.com/watch?v=Hvc226NRmSQ&t=15s
8,wat het is,3928,3930,mensen willen aansluiten nou weet je wat het i...,lApJy0bJNk0,https://www.youtube.com/watch?v=lApJy0bJNk0&t=...
9,waar ik ben,116,118,waar ik ben ik zou mijn fiets moeten en ik ga ...,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...


In [43]:
df_word_group_time_loc_threegram_result.to_excel(f"{lang_folder.capitalize()}_Threegram_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx", index=False) 

In [20]:
df_word_search_result = word_group_youtube(df_youtube_sentence, word_list, "sentence", 5)  # (sample_num+1)
df_word_search_result

,search_string,start_time,end_time,sentence,video_id
0,ik,17.40,24.04,we zijn een tijdje geleden opgericht maar wat ...,8jmOuLksBJ0
1,ik,2213.20,2220.18,ik zal ophouden over de oorzaak van die schaar...,QAjt9liE_a4
2,ik,547.96,551.36,ik vertel het verhaal aan tom en we passen de ...,HKCKdkGrazU
3,ik,38.12,45.08,ik denk dat dat de maatregelen zijn die je nie...,KgYv5wjScaM
4,ik,1057.24,1058.32,ja dat denk ik wel,gFA9Q_RrOVo
...,...,...,...,...,...
995,neem,1030.94,1034.26,neem maar mee,Gj1X8jth_Bw
996,neem,311.56,316.39,op stoom geholpen dus mijn advies neem de tijd...,lYKUmLypP5s
997,neem,230.60,233.16,ik neem de volgende dan ga ik deze,ZqkfywfGIIY
998,neem,48.42,52.60,en neem gerust een dekentje mee extra gezellig...,ELg3rJ5NeRk


In [21]:
df_word_group_time_loc_word_result = word_group_time_loc(df_word_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_word_result

,search_string,start_time,end_time,sentence,video_id
0,ik,19.829268,20.045203,we zijn een tijdje geleden opgericht maar wat ...,8jmOuLksBJ0
1,ik,2213.200000,2213.435281,ik zal ophouden over de oorzaak van die schaar...,QAjt9liE_a4
2,ik,547.960000,548.132881,ik vertel het verhaal aan tom en we passen de ...,HKCKdkGrazU
3,ik,38.120000,38.318857,ik denk dat dat de maatregelen zijn die je nie...,KgYv5wjScaM
4,ik,1057.900000,1058.140000,ja dat denk ik wel,gFA9Q_RrOVo
...,...,...,...,...,...
995,neem,1030.940000,1032.216923,neem maar mee,Gj1X8jth_Bw
996,neem,313.903971,314.330147,op stoom geholpen dus mijn advies neem de tijd...,lYKUmLypP5s
997,neem,230.750588,231.202353,ik neem de volgende dan ga ik deze,ZqkfywfGIIY
998,neem,48.518353,48.813412,en neem gerust een dekentje mee extra gezellig...,ELg3rJ5NeRk


In [22]:
df_word_group_time_loc_word_result.start_time = df_word_group_time_loc_word_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_word_result.end_time = df_word_group_time_loc_word_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_word_result

,search_string,start_time,end_time,sentence,video_id
0,ik,19.229268,20.645203,we zijn een tijdje geleden opgericht maar wat ...,8jmOuLksBJ0
1,ik,2212.600000,2214.035281,ik zal ophouden over de oorzaak van die schaar...,QAjt9liE_a4
2,ik,547.360000,548.732881,ik vertel het verhaal aan tom en we passen de ...,HKCKdkGrazU
3,ik,37.520000,38.918857,ik denk dat dat de maatregelen zijn die je nie...,KgYv5wjScaM
4,ik,1057.300000,1058.740000,ja dat denk ik wel,gFA9Q_RrOVo
...,...,...,...,...,...
995,neem,1030.340000,1032.816923,neem maar mee,Gj1X8jth_Bw
996,neem,313.303971,314.930147,op stoom geholpen dus mijn advies neem de tijd...,lYKUmLypP5s
997,neem,230.150588,231.802353,ik neem de volgende dan ga ik deze,ZqkfywfGIIY
998,neem,47.918353,49.413412,en neem gerust een dekentje mee extra gezellig...,ELg3rJ5NeRk


In [23]:
df_word_group_time_loc_word_result.start_time = df_word_group_time_loc_word_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_word_result.end_time = df_word_group_time_loc_word_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_word_result 

,search_string,start_time,end_time,sentence,video_id
0,ik,19,21,we zijn een tijdje geleden opgericht maar wat ...,8jmOuLksBJ0
1,ik,2213,2214,ik zal ophouden over de oorzaak van die schaar...,QAjt9liE_a4
2,ik,547,549,ik vertel het verhaal aan tom en we passen de ...,HKCKdkGrazU
3,ik,38,39,ik denk dat dat de maatregelen zijn die je nie...,KgYv5wjScaM
4,ik,1057,1059,ja dat denk ik wel,gFA9Q_RrOVo
...,...,...,...,...,...
995,neem,1030,1033,neem maar mee,Gj1X8jth_Bw
996,neem,313,315,op stoom geholpen dus mijn advies neem de tijd...,lYKUmLypP5s
997,neem,230,232,ik neem de volgende dan ga ik deze,ZqkfywfGIIY
998,neem,48,49,en neem gerust een dekentje mee extra gezellig...,ELg3rJ5NeRk


In [24]:
df_word_group_time_loc_word_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_word_result['video_id'].map(str)+"&t="+df_word_group_time_loc_word_result['start_time'].map(str)+"s"
df_word_group_time_loc_word_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,ik,19,21,we zijn een tijdje geleden opgericht maar wat ...,8jmOuLksBJ0,https://www.youtube.com/watch?v=8jmOuLksBJ0&t=19s
1,ik,2213,2214,ik zal ophouden over de oorzaak van die schaar...,QAjt9liE_a4,https://www.youtube.com/watch?v=QAjt9liE_a4&t=...
2,ik,547,549,ik vertel het verhaal aan tom en we passen de ...,HKCKdkGrazU,https://www.youtube.com/watch?v=HKCKdkGrazU&t=...
3,ik,38,39,ik denk dat dat de maatregelen zijn die je nie...,KgYv5wjScaM,https://www.youtube.com/watch?v=KgYv5wjScaM&t=38s
4,ik,1057,1059,ja dat denk ik wel,gFA9Q_RrOVo,https://www.youtube.com/watch?v=gFA9Q_RrOVo&t=...
...,...,...,...,...,...,...
995,neem,1030,1033,neem maar mee,Gj1X8jth_Bw,https://www.youtube.com/watch?v=Gj1X8jth_Bw&t=...
996,neem,313,315,op stoom geholpen dus mijn advies neem de tijd...,lYKUmLypP5s,https://www.youtube.com/watch?v=lYKUmLypP5s&t=...
997,neem,230,232,ik neem de volgende dan ga ik deze,ZqkfywfGIIY,https://www.youtube.com/watch?v=ZqkfywfGIIY&t=...
998,neem,48,49,en neem gerust een dekentje mee extra gezellig...,ELg3rJ5NeRk,https://www.youtube.com/watch?v=ELg3rJ5NeRk&t=48s


In [25]:
df_word_group_time_loc_word_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx", index=False)

#### Copy Move And Delete

In [26]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx")
output_file

['Dutch_200_Word_1_Youtube_0.6s_Timeshift_For_Talk_Time_Result.xlsx']

In [27]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [28]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass